In [28]:
# Transformers installation
!pip install transformers datasets pandas matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 65.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 43.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.8 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda"
model_id = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [12]:
def calculatePerplexity(input_text, output_text):
    # Tokenize the input and output texts
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    output_ids = tokenizer.encode(output_text, return_tensors="pt").to(device)
    # Concatenate the input and output token IDs TODO test
    input_output_ids = torch.cat((input_ids, output_ids), dim=-1).to(device)
    trg_len = len(output_ids)
    target_ids = input_output_ids.clone()
    target_ids[:, :-trg_len] = -100  #wtf does this do
    
    with torch.no_grad():
        outputs = model(input_output_ids, labels=target_ids)
        neg_log_likelihood = outputs.loss
        
    print(output_text +" :" + str(torch.exp(neg_log_likelihood).item()))
    return neg_log_likelihood.item()

input1 = "The quick brown fox"
input2 = "An apple a day, "
calculatePerplexity(input1, "Moves quick brown fox to")

Moves quick brown fox to :114.7151107788086
beats your mom :88.05709075927734
jumps over the lazy dog :1.0728567838668823
keeps the doctor away :1.1327600479125977
keeps the doctor dead haha wrecked :4.6016764640808105
Moves quick brown fox to :152.12586975097656
beats your mom :434.2688293457031
jumps over the lazy dog :1.9120936393737793
keeps the doctor away :1.02092707157135
keeps the doctor dead haha wrecked :5.407622337341309


1.6878094673156738

In [30]:
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer("The quick brown fox", return_tensors="pt").input_ids.to(device)

outputs_array = []

for i in range(1000):
    outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    outputs_array.append(decoded_output[0])

print(outputs_array)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

['The quick brown fox jumped over the lazy dog\n\nI think everyone has been writing on this blog long enough to have encountered that age old saying – “the quick brown fox jumped over the lazy dog”. It is a great example of how a string of words will only ever be read correctly, if every letter of every word can be clearly identified and the punctuation is understood. This is the only way that someone can understand what the words are meant to say. The words are not words unless they are understood and', 'The quick brown fox jumps over the lazy dog. See?\n\nToday I am going to go on record and say the following statement:\n\nI am, unequivocally, completely, positively and in all seriousness, officially out-of-love with blogging and all related Internet-y stuff.\n\nI know, I know. You all thought I was in-love, but really? I was just playing.\n\nThe truth is: I’m done with', 'The quick brown fox jumps over the lazy dog.\n\nThere is no question that technology has dramatically increased 

#Mass Calculation

In [31]:
y = []
for i in outputs_array:
    y.append(calculatePerplexity("The quick brown fox", i))

import matplotlib.pyplot as plt

# Assuming you have already calculated the perplexity values and stored them in the 'y' list
plt.figure(figsize=(8, 6))
plt.hist(y, bins=100, edgecolor='black')
plt.xlabel('Perplexity')
plt.ylabel('Frequency')
plt.title('Distribution of Perplexity Values')
plt.grid(True)
plt.show()

The quick brown fox jumped over the lazy dog

I think everyone has been writing on this blog long enough to have encountered that age old saying – “the quick brown fox jumped over the lazy dog”. It is a great example of how a string of words will only ever be read correctly, if every letter of every word can be clearly identified and the punctuation is understood. This is the only way that someone can understand what the words are meant to say. The words are not words unless they are understood and :11.3235502243042
The quick brown fox jumps over the lazy dog. See?

Today I am going to go on record and say the following statement:

I am, unequivocally, completely, positively and in all seriousness, officially out-of-love with blogging and all related Internet-y stuff.

I know, I know. You all thought I was in-love, but really? I was just playing.

The truth is: I’m done with :5.8241658210754395
The quick brown fox jumps over the lazy dog.

There is no question that technology has drama

#Load data to/from CSV

In [40]:
import pandas as pd
# Create a DataFrame using the prompts and completions
prompts= ["The quick brown fox" for x in range(1000)]
completions = outputs_array
perplexity = y
data = {'prompt': prompts, 'completions': completions, 'perplexity': perplexity}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('perplexity_test1.csv', index=False)

print("Prompts and completions have been saved to 'prompts_completions.csv'.")

Prompts and completions have been saved to 'prompts_completions.csv'.


In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('perplexity_test1.csv')

# Extract the prompts array from the DataFrame
prompts = df['prompt'].tolist()

